In [1]:
import pandas as pd
import os
import os.path as op
import h5py
import numpy as np
import matplotlib.pyplot as plt
from dtw import *
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from scipy.interpolate import interpn
from tqdm.notebook import trange, tqdm
from scipy.ndimage import gaussian_filter1d
from mne.stats import permutation_cluster_test
from matplotlib import colors
from matplotlib.colors import Normalize 
from matplotlib import cm
import scipy.io
from mne.io import read_epochs_eeglab

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
pipeline='NEARICA_behav'
ages=['9m','12m','adult']

cluster_chans=['E16', 'E20', 'E21', 'E22','E41', 'E49', 'E50', 'E51']

def load_bursts(pipeline, age):
    bursts = {
        'age': [],
        'subject': [],
        'chan': [],
        'cluster': [],
        'epoch': [],
        'fwhm_freq': [],
        'fwhm_time': [],
        'peak_adjustment': [],
        'peak_amp_base': [],
        'peak_amp_iter': [],
        'peak_freq': [],
        'peak_time': [],
        'polarity': [],
        'trial': [],
        'waveform': [],        
        'waveform_times': []
    }

    subjects=pd.read_csv(op.join('/home/bonaiuto/dev_beta_umd/data',age,'data/participants.tsv'), sep='\t')
    for subject in subjects['participant_id']:
        print('loading {}'.format(subject))
        fname=op.join('/home/bonaiuto/dev_beta_umd/data',age,'derivatives',pipeline,subject,'processed_data/processed_bursts.mat')
        if os.path.exists(fname):
            with h5py.File(fname, 'r') as f:
                bursts['age'].extend([age for x in list(f['bursts']['trial'])])
                bursts['subject'].extend([subject for x in list(f['bursts']['trial'])])
                bursts['chan'].extend(list(np.squeeze(f['bursts']['chan'][:])))
                bursts['cluster'].extend([u''.join(chr(c) for c in np.squeeze(f[obj_ref][:])) for obj_ref in np.squeeze(f['bursts']['cluster'][:])])
                bursts['epoch'].extend([u''.join(chr(c) for c in np.squeeze(f[obj_ref][:])) for obj_ref in np.squeeze(f['bursts']['epoch'][:])])
                bursts['fwhm_freq'].extend(list(np.squeeze(f['bursts']['fwhm_freq'][:])))
                bursts['fwhm_time'].extend(list(np.squeeze(f['bursts']['fwhm_time'][:])))
                bursts['peak_adjustment'].extend(list(np.squeeze(f['bursts']['peak_adjustment'][:])))
                bursts['peak_amp_base'].extend(list(np.squeeze(f['bursts']['peak_amp_base'][:])))
                bursts['peak_amp_iter'].extend(list(np.squeeze(f['bursts']['peak_amp_iter'][:])))
                bursts['peak_freq'].extend(list(np.squeeze(f['bursts']['peak_freq'][:])))
                bursts['peak_time'].extend(list(np.squeeze(f['bursts']['peak_time'][:])))
                bursts['polarity'].extend(list(np.squeeze(f['bursts']['polarity'][:])))
                bursts['trial'].extend(list(np.squeeze(f['bursts']['trial'][:])))
                for i in range(f['bursts']['waveform'][:,:].shape[1]):
                    bursts['waveform'].append(f['bursts']['waveform'][:,i])
                bursts['waveform_times']=f['bursts']['waveform_times'][:]
                
    bursts['age']=np.array(bursts['age'])
    bursts['subject']=np.array(bursts['subject'])
    bursts['chan']=np.array(bursts['chan'])
    bursts['cluster']=np.array(bursts['cluster'])
    bursts['epoch']=np.array(bursts['epoch'])
    bursts['fwhm_freq']=np.array(bursts['fwhm_freq'])
    bursts['fwhm_time']=np.array(bursts['fwhm_time'])
    bursts['peak_adjustment']=np.array(bursts['peak_adjustment'])
    bursts['peak_amp_base']=np.array(bursts['peak_amp_base'])
    bursts['peak_amp_iter']=np.array(bursts['peak_amp_iter'])
    bursts['peak_freq']=np.array(bursts['peak_freq'])
    bursts['peak_time']=np.array(bursts['peak_time'])
    bursts['polarity']=np.array(bursts['polarity'])
    bursts['trial']=np.array(bursts['trial'])
    bursts['waveform']=np.array(bursts['waveform'])
    bursts['waveform_times']=np.array(bursts['waveform_times'])
    
    return bursts

In [4]:
c_bursts_9m=load_bursts(pipeline, '9m')

loading sub-010
loading sub-012
loading sub-013
loading sub-014
loading sub-015
loading sub-016
loading sub-018
loading sub-019
loading sub-020
loading sub-021
loading sub-023
loading sub-024
loading sub-025
loading sub-026
loading sub-027
loading sub-028
loading sub-029
loading sub-030
loading sub-031
loading sub-032
loading sub-033
loading sub-036
loading sub-037
loading sub-038
loading sub-041
loading sub-042
loading sub-043
loading sub-044


In [5]:
c_bursts_12m=load_bursts(pipeline, '12m')

loading sub-001
loading sub-002
loading sub-003
loading sub-004
loading sub-005
loading sub-006
loading sub-007
loading sub-008
loading sub-009
loading sub-010
loading sub-011
loading sub-012
loading sub-013
loading sub-014
loading sub-015
loading sub-016
loading sub-017
loading sub-018
loading sub-019
loading sub-020
loading sub-021
loading sub-022
loading sub-023
loading sub-024
loading sub-025
loading sub-026
loading sub-027
loading sub-028
loading sub-029
loading sub-030
loading sub-031
loading sub-032
loading sub-033


In [6]:
c_bursts_adult=load_bursts(pipeline, 'adult')

loading sub-001
loading sub-002
loading sub-003
loading sub-004
loading sub-005
loading sub-006
loading sub-007
loading sub-008
loading sub-009
loading sub-010
loading sub-011
loading sub-012
loading sub-013
loading sub-014
loading sub-015
loading sub-016
loading sub-017
loading sub-018
loading sub-019
loading sub-020
loading sub-021
loading sub-022


In [8]:
c_burst_times_9m=c_bursts_9m['waveform_times']
c_burst_times_12m=c_bursts_12m['waveform_times']
c_burst_times_adult=c_bursts_adult['waveform_times']
med_c_9m=np.percentile(c_bursts_9m['waveform'], 50, axis=0)
med_c_12m=np.percentile(c_bursts_12m['waveform'], 50, axis=0)
med_c_adult=np.percentile(c_bursts_adult['waveform'], 50, axis=0)
norm_c_adult=med_c_adult/np.max(med_c_adult)
norm_c_12m=med_c_12m/np.max(med_c_12m)
norm_c_9m=med_c_9m/np.max(med_c_9m)
alignment_c_9m=dtw(norm_c_9m, norm_c_adult, keep_internals=True, step_pattern=rabinerJuangStepPattern(5,"c"))
aligned_c_9m_idx=warp(alignment_c_9m,index_reference=False)
alignment_c_12m=dtw(norm_c_12m, norm_c_adult, keep_internals=True, step_pattern=rabinerJuangStepPattern(5,"c"))
aligned_c_12m_idx=warp(alignment_c_12m,index_reference=False)
aligned_c_9m_bursts=c_bursts_9m['waveform'][:,aligned_c_9m_idx]
aligned_c_12m_bursts=c_bursts_12m['waveform'][:,aligned_c_12m_idx]
aligned_c_adult_bursts=c_bursts_adult['waveform'][:,:-1]

aligned_c_bursts=np.vstack([aligned_c_9m_bursts,aligned_c_12m_bursts,aligned_c_adult_bursts])

In [9]:
c_pca = PCA(n_components=20)
burst_std_c_9m=RobustScaler().fit_transform(aligned_c_9m_bursts)
burst_std_c_12m=RobustScaler().fit_transform(aligned_c_12m_bursts)
burst_std_c_adult=RobustScaler().fit_transform(aligned_c_adult_bursts)
burst_std_c=np.vstack([burst_std_c_9m, burst_std_c_12m, burst_std_c_adult])
c_pca=c_pca.fit(burst_std_c)
pcs_c_9m = c_pca.transform(burst_std_c_9m)
pcs_c_12m = c_pca.transform(burst_std_c_12m)
pcs_c_adult = c_pca.transform(burst_std_c_adult)

pca_components_c_9m = pd.DataFrame(pcs_c_9m)
pca_components_c_12m = pd.DataFrame(pcs_c_12m)
pca_components_c_adult = pd.DataFrame(pcs_c_adult)

pcs_c=np.vstack([pcs_c_9m,pcs_c_12m,pcs_c_adult])

In [10]:
pca_c_9m_dict={i:c_bursts_9m[i] for i in c_bursts_9m if i!='waveform' and i!='waveform_times'}
for i in range(20):
    pca_c_9m_dict['PC{}'.format(i+1)]=pcs_c_9m[:,i]
df_c_9m=pd.DataFrame.from_dict(pca_c_9m_dict,orient='index').transpose()

In [11]:
pca_c_12m_dict={i:c_bursts_12m[i] for i in c_bursts_12m if i!='waveform' and i!='waveform_times'}
for i in range(20):
    pca_c_12m_dict['PC{}'.format(i+1)]=pcs_c_12m[:,i]
df_c_12m=pd.DataFrame.from_dict(pca_c_12m_dict,orient='index').transpose()

In [12]:
pca_c_adult_dict={i:c_bursts_adult[i] for i in c_bursts_adult if i!='waveform' and i!='waveform_times'}
for i in range(20):
    pca_c_adult_dict['PC{}'.format(i+1)]=pcs_c_adult[:,i]
df_c_adult=pd.DataFrame.from_dict(pca_c_adult_dict,orient='index').transpose()

In [13]:
df=pd.concat([df_c_9m,df_c_12m,df_c_adult], ignore_index=True, sort=False)
df.to_csv('/home/bonaiuto/dev_beta_umd/output/bursts.csv')